In [1]:
import websocket
import json
import zlib
try:
    import thread
except ImportError:
    import _thread as thread
import time
import pickle
import datetime,os

import win32com.client
import re

import queue

In [2]:
websocket_url='wss://ks-live-dmcmt-bj6-pm-01.chat.bilibili.com/sub'

send1=[ 0x00, 0x00, 0x00, 0x6D, 0x00, 0x10, 0x00, 0x01, 0x00, 0x00, 0x00, 0x07, 0x00, 0x00, 0x00, 0x01, 0x7B, 0x22, 0x75, 0x69, 0x64, 0x22, 0x3A, 0x32, 0x38, 0x34, 0x39, 0x36, 0x34, 0x37, 0x37, 0x2C, 0x22, 0x72, 0x6F, 0x6F, 0x6D, 0x69, 0x64, 0x22, 0x3A, 0x31, 0x34, 0x37, 0x30, 0x37, 0x34, 0x39, 0x35, 0x2C, 0x22, 0x70, 0x72, 0x6F, 0x74, 0x6F, 0x76, 0x65, 0x72, 0x22, 0x3A, 0x32, 0x2C, 0x22, 0x70, 0x6C, 0x61, 0x74, 0x66, 0x6F, 0x72, 0x6D, 0x22, 0x3A, 0x22, 0x77, 0x65, 0x62, 0x22, 0x2C, 0x22, 0x63, 0x6C, 0x69, 0x65, 0x6E, 0x74, 0x76, 0x65, 0x72, 0x22, 0x3A, 0x22, 0x31, 0x2E, 0x36, 0x2E, 0x33, 0x22, 0x2C, 0x22, 0x74, 0x79, 0x70, 0x65, 0x22, 0x3A, 0x32, 0x7D ]
send2=[ 0x00, 0x00, 0x00, 0x1F, 0x00, 0x10, 0x00, 0x01, 0x00, 0x00, 0x00, 0x02, 0x00, 0x00, 0x00, 0x01, 0x5B, 0x6F, 0x62, 0x6A, 0x65, 0x63, 0x74, 0x20, 0x4F, 0x62, 0x6A, 0x65, 0x63, 0x74, 0x5D ]

In [3]:
user_gift_dir='D:/OneDrive/其他/直播相关/program/database/user_gift.pkl'
user_name_speak_dir='D:/OneDrive/其他/直播相关/program/database/user_name_speak.pkl'
log_dir='D:\OneDrive\其他\直播相关\program\log.txt'
def name_dir(i):
    return 'D:\OneDrive\其他\直播相关\program\\name_{}.txt'.format(i)

In [4]:
def decode_json(msg):
    return json.loads(msg[16:])
def decomp(msg):
    try:
        return zlib.decompress(msg[16:])
    except Exception:
        print('error in decomp : {}'.format(msg[16:]))

def decode_msg(msg,debug=False):
    if debug:
        print(msg)
    if len(msg)==16:
        return []
    if len(msg)==20:
        print('当前观众：{}'.format(int.from_bytes(msg[16:], byteorder='big')))
        return []
    try:
        return [decode_json(msg)]
    except Exception:
        temp = decomp(msg)
#         print(split_jsons(temp))
        return [json.loads(i) for i in split_jsons(temp)]
    
def split_jsons(test_msg_4):
    activate=1
    stack=0
    start=[]
    end=[]
    escape=0
    i=16
    while i<len(test_msg_4):
#         print('i={},msg[i]={},stack={}'.format(i,bytes([test_msg_4[i]]),stack))
        if test_msg_4[i]==34 and escape%2!=1:
            activate=1-activate
            i+=1
            continue
        if test_msg_4[i]==92:
            escape+=1
        else:
            escape=0
        if activate:
            if test_msg_4[i]==123:
                if stack==0:
                    start.append(i)
                stack+=1
            if test_msg_4[i]==125:
                stack-=1
                if stack==0:
                    end.append(i)
                    i+=16
        i+=1
    result=[]
    for i2 in range(len(start)):
        try:
            result.append(test_msg_4[start[i2]:end[i2]+1])
        except Exception:
            print('error while append bytes to result at split_jsons.')
            print('start & end : {},{}'.format(start,end))
            print('msg : \n{}'.format(test_msg_4))
    return result